In [1]:
import QuantLib as ql
import pandas as pd
import xlwings as xw
import numpy as np

In [2]:
desired_width = 320
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 200)

## 1. IRS 정보

- 발행일 : 2024.8.31
- 만기일 : 2029.8.31
- 고정금리 : 3.5%, SemmiAnnual, ActualActual 
- 변동금리 : 1yr ref rate + 35bps, Annual, Acutal360
- type : Fixed Rate Receiver (Buy Position)
- 액면가 : 1_000_000

In [4]:
currency = ql.KRWCurrency()
issueDate = ql.Date(31,8,2024)
maturityDate = ql.Date(31,8,2029)
faceval = 1_000_000

## 2. YieldTermStructure Handle

In [5]:
valDate = ql.Date(31,8,2025)
ql.Settings.instance().evaluationDate = valDate

In [6]:
def load_from_xls(xls:str,sht_nm:str,rng:str):
    app = xw.App(visible=False)
    try:
        wb_0 = xw.Book(xls)
        sht = wb_0.sheets[sht_nm]
        df = sht.range(rng).expand().options(pd.DataFrame,index=False).value
        wb_0.close()
    except:
        return 1
    #wb_0.close()
    app.kill()
    return df

In [7]:
df_ = load_from_xls("C:\work\youtube_lecture\lecture_0\FSS_IFRS17 및 K-ICS 금리기간구조(원화)__25.8.xlsm","원화 무위험 금리기간구조","AB12")

In [8]:
ql_date_list = list(ql.MakeSchedule(ql.Date(31,8,2025),ql.Date(31,8,2125), ql.Period("1M"),endOfMonth=True))
discount_curve = ql.DiscountCurve(ql_date_list, df_.iloc[:,1], ql.ActualActual(ql.ActualActual.ISDA), ql.SouthKorea(ql.SouthKorea.Settlement),ql.LogLinear())

In [10]:
yld_handle = ql.RelinkableYieldTermStructureHandle(discount_curve)
spread = ql.SimpleQuote(0.0)
yld_curve = ql.ZeroSpreadedTermStructure(yld_handle,ql.QuoteHandle(spread))
yld_handle_1 = ql.RelinkableYieldTermStructureHandle(yld_curve)

# 3. Fixed Leg

In [11]:
calendar = ql.SouthKorea(ql.SouthKorea.Settlement)
fixed_leg_tenor = ql.Period(ql.Semiannual)
bdc = ql.ModifiedFollowing

In [12]:
fixed_schedule = ql.Schedule(issueDate, maturityDate,
                                     fixed_leg_tenor, calendar,
                                     bdc, bdc,
                                     ql.DateGeneration.Backward, False)

In [13]:
list(fixed_schedule)

[Date(30,8,2024),
 Date(28,2,2025),
 Date(29,8,2025),
 Date(27,2,2026),
 Date(31,8,2026),
 Date(26,2,2027),
 Date(31,8,2027),
 Date(29,2,2028),
 Date(31,8,2028),
 Date(28,2,2029),
 Date(31,8,2029)]

In [14]:
fixed_day_count = ql.ActualActual(ql.ActualActual.ISDA)
fixed_rate = 3.5/100

# 4.Floating Leg

In [15]:
float_leg_tenor = ql.Period(ql.Annual)

In [16]:
float_schedule = ql.Schedule(issueDate, maturityDate,
                                     float_leg_tenor, calendar,
                                     bdc, bdc,
                                     ql.DateGeneration.Backward, False)

In [17]:
float_day_count = ql.Actual360()

In [18]:
ref_index = ql.IborIndex("KRW_int_", ql.Period("1Y"), 0, currency, calendar,
                                 ql.Following, False, float_day_count, yld_handle_1)

In [19]:
float_spread = 35/10000

In [20]:
np.array(float_schedule)

array([Date(30,8,2024), Date(29,8,2025), Date(31,8,2026), Date(31,8,2027), Date(31,8,2028), Date(31,8,2029)], dtype=object)

In [21]:
fixing_rate = 2.5/100

In [22]:
for date_ in list(float_schedule):

    fixing_date = calendar.advance(date_, ql.Period(-0, ql.Days))

    if fixing_date <= valDate:
        ref_index.addFixing(fixing_date,fixing_rate)


# 5. Interest Rate Swwap Setting

In [23]:
swap_type = ql.VanillaSwap.Receiver

In [24]:
swap_obj = ql.VanillaSwap(swap_type, faceval, 
                      fixed_schedule,fixed_rate, fixed_day_count, 
                      float_schedule,ref_index, float_spread, float_day_count)

# 6 Pricing Engine 구성 및 연결

In [25]:
swap_engine = ql.DiscountingSwapEngine(yld_handle_1)

In [26]:
swap_obj.setPricingEngine(swap_engine)

# 7. 공정가치의 계산

In [27]:
swap_obj.NPV()

21854.69697447488

# 8. 현금흐름의 검증

In [28]:
data = []
for cf0 in list(map(ql.as_fixed_rate_coupon, swap_obj.leg(0)))[:-1]:
    row = {"date": cf0.date().ISO(),
           "accrualStartDate": cf0.accrualStartDate().ISO(),
           "accrualEndDate": cf0.accrualEndDate().ISO(),
           "accrualDays": cf0.accrualDays(),
           "accrualPeriod": cf0.accrualPeriod(),
           "rate": cf0.rate(),
           "amount": cf0.amount(),
           "nominal": cf0.nominal(),
           "dayCounter": str(cf0.dayCounter()),
           "InterestRate": str(cf0.interestRate()),
           }
    data.append(row)

df_cf_fixed = pd.DataFrame(data)
df_cf_fixed

,date,accrualStartDate,accrualEndDate,accrualDays,accrualPeriod,rate,amount,nominal,dayCounter,InterestRate
0,2025-02-28,2024-08-30,2025-02-28,182,0.497702,0.035,17419.567333,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
1,2025-08-29,2025-02-28,2025-08-29,182,0.498630,0.035,17452.054795,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
2,2026-02-27,2025-08-29,2026-02-27,182,0.498630,0.035,17452.054795,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
3,2026-08-31,2026-02-27,2026-08-31,185,0.506849,0.035,17739.726027,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
4,2027-02-26,2026-08-31,2027-02-26,179,0.490411,0.035,17164.383562,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
5,2027-08-31,2027-02-26,2027-08-31,186,0.509589,0.035,17835.616438,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
6,2028-02-29,2027-08-31,2028-02-29,182,0.498188,0.035,17436.597051,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
7,2028-08-31,2028-02-29,2028-08-31,184,0.502732,0.035,17595.628415,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...
8,2029-02-28,2028-08-31,2029-02-28,181,0.494970,0.035,17323.938918,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...


In [31]:
data_1 = []
for cf1 in list(map(ql.as_floating_rate_coupon, swap_obj.leg(1)))[:-1]:
    row = {"date": cf1.date().ISO(),
           "fixingDate": cf1.fixingDate().ISO(),
           "accrualStartDate": cf1.accrualStartDate().ISO(),
           "accrualEndDate": cf1.accrualEndDate().ISO(),
           "accrualDays": cf1.accrualDays(),
           "accrualPeriod": cf1.accrualPeriod(),
           "gearing": cf1.gearing(),
           "forward": cf1.indexFixing(),
           "spread": cf1.spread(),
           "rate": cf1.rate(),
           "nominal": cf1.nominal(),
           "amount": cf1.amount(),
           "dayCounter": str(cf1.dayCounter()),
           }
    data_1.append(row)

df_cf_float = pd.DataFrame(data_1)
df_cf_float

,date,fixingDate,accrualStartDate,accrualEndDate,accrualDays,accrualPeriod,gearing,forward,spread,rate,nominal,amount,dayCounter
0,2025-08-29,2024-08-30,2024-08-30,2025-08-29,364,1.011111,1.0,0.025000,0.0035,0.028500,1000000.0,28816.666667,Actual/360 day counter
1,2026-08-31,2025-08-29,2025-08-29,2026-08-31,367,1.019444,1.0,0.025000,0.0035,0.028500,1000000.0,29054.166667,Actual/360 day counter
2,2027-08-31,2026-08-31,2026-08-31,2027-08-31,365,1.013889,1.0,0.024357,0.0035,0.027857,1000000.0,28243.412047,Actual/360 day counter
3,2028-08-31,2027-08-31,2027-08-31,2028-08-31,366,1.016667,1.0,0.025628,0.0035,0.029128,1000000.0,29613.790640,Actual/360 day counter


In [32]:
df_cf_fixed["leg"] = 'Fix'
df_cf_float["leg"] = 'Float'
df_cf = pd.concat([df_cf_fixed,df_cf_float],axis=0)
df_cf

,date,accrualStartDate,accrualEndDate,accrualDays,accrualPeriod,rate,amount,nominal,dayCounter,InterestRate,leg,fixingDate,gearing,forward,spread
0,2025-02-28,2024-08-30,2025-02-28,182,0.497702,0.035000,17419.567333,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
1,2025-08-29,2025-02-28,2025-08-29,182,0.498630,0.035000,17452.054795,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
2,2026-02-27,2025-08-29,2026-02-27,182,0.498630,0.035000,17452.054795,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
3,2026-08-31,2026-02-27,2026-08-31,185,0.506849,0.035000,17739.726027,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
4,2027-02-26,2026-08-31,2027-02-26,179,0.490411,0.035000,17164.383562,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
5,2027-08-31,2027-02-26,2027-08-31,186,0.509589,0.035000,17835.616438,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
6,2028-02-29,2027-08-31,2028-02-29,182,0.498188,0.035000,17436.597051,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
7,2028-08-31,2028-02-29,2028-08-31,184,0.502732,0.035000,17595.628415,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
8,2029-02-28,2028-08-31,2029-02-28,181,0.494970,0.035000,17323.938918,1000000.0,Actual/Actual (ISDA) day counter,3.500000 % Actual/Actual (ISDA) simple compoun...,Fix,NaN,NaN,NaN,NaN
0,2025-08-29,2024-08-30,2025-08-29,364,1.011111,0.028500,28816.666667,1000000.0,Actual/360 day counter,NaN,Float,2024-08-30,1.0,0.025000,0.0035


# 9. 현재가치검증¶

In [33]:
data_0 = []
for cf0 in swap_obj.leg(0):
    row1 = {"date": cf0.date().ISO(),
            "CFamount": cf0.amount()}
    if cf0.date() <= valDate:
        row1["yearFrac"] = 0
        row1["discfac"] = 0
    else:
        row1["yearFrac"] = fixed_day_count.yearFraction(valDate, cf0.date())
        row1["discfac"] = yld_handle_1.discount(cf0.date())

    row1["pv"] = row1["CFamount"] * row1["discfac"]
    data_0.append(row1)

df_fixed_pv = pd.DataFrame(data_0)
pv_sum_fixed = df_fixed_pv.pv.sum()

data_1 = []
for cf in swap_obj.leg(1):
    row1 = {"date": cf.date().ISO(),
            "CFamount": cf.amount()}
    if cf.date() <= valDate:
        row1["yearFrac"] = 0
        row1["discfac"] = 0
    else:
        row1["yearFrac"] = fixed_day_count.yearFraction(valDate, cf.date())
        row1["discfac"] = yld_handle_1.discount(cf.date())

    row1["pv"] = row1["CFamount"] * row1["discfac"]
    data_1.append(row1)

df_float_pv = pd.DataFrame(data_1)
pv_sum_float = df_float_pv.pv.sum()

In [34]:
df_fixed_pv

,date,CFamount,yearFrac,discfac,pv
0,2025-02-28,17419.567333,0.000000,0.000000,0.000000
1,2025-08-29,17452.054795,0.000000,0.000000,0.000000
2,2026-02-27,17452.054795,0.493151,0.988449,17250.472305
3,2026-08-31,17739.726027,1.000000,0.977918,17348.005393
4,2027-02-26,17164.383562,1.490411,0.966173,16583.768992
5,2027-08-31,17835.616438,2.000000,0.954351,17021.437387
6,2028-02-29,17436.597051,2.498188,0.942055,16426.236549
7,2028-08-31,17595.628415,3.000921,0.930116,16365.981459
8,2029-02-28,17323.938918,3.495890,0.917868,15901.094237
9,2029-08-31,17643.835616,4.000000,0.905226,15971.653131


In [35]:
pv_sum_fixed

np.float64(132868.6494533065)

In [36]:
pv_sum_fixed - pv_sum_float

np.float64(21854.69697447488)

# 10. Effective Duration

In [37]:
npv_base = swap_obj.NPV()

In [38]:
spread.setValue(10/10000)
npv_up = swap_obj.NPV()

In [39]:
spread.setValue(-10/10000)
npv_dn = swap_obj.NPV()

In [40]:
eff_dur =  (npv_dn - npv_up) / (2*npv_base * 0.001)
eff_dur

131.7458497010906